### RNN模型总览

RNN 是一种循环神经网络，用于序列模型的预测

在 RNN 中，使用一种叫做潜变量的东西来存储历史序列信息

在将历史信息都录入到潜变量中后，再通过潜变量预测后面一个 token

得到的输出是预测的 token 是词表中任何一个 token 的概率（softmax 后）

![](md-img/rnn-model.jpg)

上图中 H 表示潜变量（保存历史状态），没有下标的原因是不需要保存每一个历史状态，直接对潜变量进行更新即可

图中每一个箭头表示一个全连接，其中使用 H 和 一个 token 来更新 H 使用的两个全连接可以使用一个偏置即可

通常使用 H 和新的 token 对 H 进行更新时，得到的是对新的 H 的打分，还可以添加激活函数，上图中省略了

每一个 token 的输入大小为 vocab_size（即使用独热编码来表示 token）

每一个预测 token 的输出大小也是 vocab_size，表示对该 token 是词表中任何一个词的打分

得分输出经过一个 softmax 层即可得到是词表中任何一个词的概率

<br>

### 训练中的输出

训练使用的样本的序列长度为指定的时间步长

预测的输出是相对于样本向后偏移一个序列号的序列

也就是说每次对潜变量进行一次更新，都会用当前的潜变量进行一次 token 的预测

<br>

### 损失函数

损失函数采用困惑度 (perplexity) 来表示，困惑度其实就是 exp(crossEntropyLoss)

<br>

### 梯度裁剪

在 RNN 中，梯度的反向传播需要进行很多次的矩阵计算，这样可能会导致矩阵爆炸（元素值过大）

梯度裁剪就是对最终得到的梯度进行比例缩放

计算梯度整体的 L2 范数，若其大于阈值 theta，则将所有的梯度乘 (theta / L2)

<br>

### 潜变量初始化

在没有历史信息的时候，潜变量可以全部初始化为 0

不同样本对应的序列不连续的话，这些样本的潜变量就不一样，需要为这些样本分配不同的潜变量

潜变量的形状可以视为 (batch_size, hiden_size)，其中 hiden_size 可以自定义

<br>

### 使用模型进行预测

使用 RNN 模型进行预测的任务一般是通过给定 prefix，预测后面可能出现的一系列 token

prefix 就是前面的 token 序列

在进行预测时，需要将 prefix 中的 token 序列信息录入到潜变量中，这个操作叫做模型的预热

然后在用潜变量进行预测输出，并利用输出继续更新潜变量。循环这个操作，理论上可以无限往后面进行预测。

<br>

### 代码从零实现

In [22]:
import torch
from torch.nn import functional


class RNNModel:
    # hiden_size 是潜变量的大小
    def __init__(self, time_step, vocab_size, hiden_size):
        self.time_step = time_step
        self.vocab_size = vocab_size
        self.hiden_size = hiden_size

        # 用于更新潜变量的模型参数
        self.w_hh = torch.randn(hiden_size, hiden_size) * 0.01
        self.w_xh = torch.randn(vocab_size, hiden_size) * 0.01
        self.b_h = torch.zeros(hiden_size)

        # 通过潜变量获取输出的模型参数
        self.w_y = torch.randn(hiden_size, vocab_size) * 0.01
        self.b_y = torch.randn(vocab_size) * 0.01

        self.w_hh.requires_grad_(True)
        self.w_xh.requires_grad_(True)
        self.b_h.requires_grad_(True)
        self.w_y.requires_grad_(True)
        self.b_y.requires_grad_(True)


    # 在没有任何历史信息的情况下，初始化潜变量（全部初始化为 0）
    def init_state(self, batch_size):
        return torch.zeros(batch_size, self.hiden_size)

    # 对一个批次的样本进行正向传播
    def __call__(self, X):
        return self.forward(X)
    
    # X 的形状为 (time_step, batch_size, vocab_size)
    # 这样可以同时更新不同样本的潜变量
    def forward(self, X):
        h = self.init_state(X.shape[1])    # 初始化潜变量
        Y = []     # 用于保存每一个时间步长上的预测值
        for x in X:
            h = h @ self.w_hh + x @ self.w_xh + self.b_h
            y = h @ self.w_y + self.b_y
            Y.append(y)
        return torch.tensor(Y), (h,)
    
    # 梯度裁剪，防止梯度爆炸，只有整体梯度的 L2 范数大于 theta 时，才需要进行裁剪
    def grad_clipping(self, theta):
        norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in self.get_parameters()))
        if norm > theta:
            for param in self.get_parameters():
                param.grad[:] *= theta / norm
    
    def get_parameters(self):
        return [self.w_hh, self.w_xh, self.b_h, self.w_y, self.b_y]
    
    # 预测函数，根据 num_prefix 个 token 的输入，预测后续 num_predic 个 token 的输出
    # prefix 的形状为 (len, vocab_size)
    def prefict(self, prefix, num_predic):
        # 潜变量的预热
        h = self.init_state(1)
        for x in prefix:
            h = h @ self.w_hh + x.reshape(1, -1) @ self.w_xh + self.b_h
        
        # 进行后续 num_predic 个词的预测
        Y = []
        for _ in range(num_predic):
            y = (h @ self.w_y + self.b_y).flatten().argmax()
            h = h @ self.w_hh + functional.one_hot(y, self.vocab_size).reshape(1, -1) @ self.w_xh + self.b_h
            Y.append(y)
        return torch.tensor(Y)

In [ ]:
# 损失函数的计算，使用困惑度 (perplexity) 来衡量损失
# y_hat 和 y 的形状都是 (time_step, batch_size, vocab_size)
# 每个样本在不同的时间步长上都有输出（vocab_size 个得分）
def loss_func(y_hat, y):
    # 将数据形状变为(time_step * batch_size, vocab_size)
    # 方便计算 softmax 和 crossEntropyLoss
    y1 = torch.cat(y_hat, dim=0)
    y2 = torch.cat(y, dim=0)
    return torch.exp(torch.nn.CrossEntropyLoss(y1, y2))

In [24]:
# 进行一次参数更新（训练一个 batch）
def train_batch(rnn_model: RNNModel, X:torch.Tensor, Y:torch.Tensor, optimizer:torch.optim.Optimizer):
    optimizer.zero_grad()
    y, _ = rnn_model.forward(X)
    loss = loss_func(y, Y)
    loss.backword()
    rnn_model.grad_clipping(1)
    optimizer.step()

然后使用不同的 batch 循环调用上面的训练即可